In [1]:
%load_ext Cython
%load_ext autoreload
%autoreload 2

from dask_image.ndfilters import median_filter, threshold_local, percentile_filter, gaussian
from cython.parallel import prange

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from datetime import datetime
from satpy import Scene
import dask.array as da
from glob import glob

from pyfires.PYF_detection import stage1_tests, run_basic_night_detection
import pyfires.PYF_Consts as PYFc
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import numpy as np

import warnings
warnings.filterwarnings('ignore')


In [ ]:
indir = 'D:/sat_data/HIM/'
odir = 'D:/sat_data/SEV_FIRE/'

indir = 'D:/sat_data/ahi_main/in/1000/'
odir = 'D:/sat_data/ahi_main/out/'

ifiles_l15 = glob(indir + '*.DAT')

ifile_cld = f'{indir}/AHI-CMSK_v1r1_h09_s202308240540213_e202308240549407_c202308240557548.nc'



rad_dict = {'vi1': 1647.567,
            'swi': 10.259}

bdict = {'vi1_band': 'B03',
         'vi2_band': 'B06',
         'mir_band': 'B07',
         'lwi_band': 'B13'}

blist = []
for item in bdict:
    blist.append(bdict[item])

scn = initial_load(ifiles_l15, 'ahi_hsd', bdict, rad_dict, mir_bt_thresh=270, lw_bt_thresh=260)
#scn.save_datasets(base_dir=odir, enhance=False, dtype=np.float32)

vi_rad_arr = np.array(scn['VI1_RAD'])
ndfi_arr = np.array(scn['mi_ndfi'])
vi_diff_arr = np.array(scn['VI1_DIFF'])
btd_arr = np.array(scn['BTD']) 
mir_bt_arr = np.array(scn['MIR__BT'])
irr_arr = np.array(scn['VI2_RAD'])
lwi_bt_arr = np.array(scn['LW1__BT'])
lwi_rad_arr = np.array(scn['LW1_RAD'])
lsm = np.array(scn['LSM'])
sza_arr = scn['SZA'].data

cannot convert float NaN to integer


In [33]:
stg1_output = stage1_tests(scn,
                           ksizes=[5, 7, 9],
                           do_lsm_mask=False)

pfp_arr = np.array(scn['PFP'])

In [5]:
st_time = datetime.utcnow()
outa = get_mea_std_window(pfp_arr,       # Potential fire pixel candidates
                          vi_rad_arr,    # VIS chan
                          ndfi_arr,      # NDFI
                          lwi_bt_arr,    # LW Brightness Temperature
                          btd_arr,       # MIR-LW BTD
                          mir_bt_arr,    # MIR BT
                          vi_diff_arr,   # MIR-LWIR-VIS radiance diff
                          lsm,           # The land-sea mask
                          15)
en_time = datetime.utcnow()

print((en_time - st_time).total_seconds())

187.413045


In [28]:

perc_good  = np.squeeze(outa[0,:,:]); n_winpix   = np.squeeze(outa[1,:,:])
n_cloudpix = np.squeeze(outa[2,:,:]); n_waterpix = np.squeeze(outa[3,:,:])
mean_lw    = np.squeeze(outa[4,:,:]); std_lw     = np.squeeze(outa[5,:,:])
mean_nd    = np.squeeze(outa[6,:,:]); std_nd     = np.squeeze(outa[7,:,:])
mean_vi    = np.squeeze(outa[8,:,:]); std_vi     = np.squeeze(outa[9,:,:])
mean_btd   = np.squeeze(outa[10,:,:]); std_btd    = np.squeeze(outa[11,:,:])
mean_mir   = np.squeeze(outa[12,:,:]); std_mir    = np.squeeze(outa[13,:,:])
mean_vid   = np.squeeze(outa[14,:,:]); std_vid    = np.squeeze(outa[15,:,:])

vis_window_arr = vi_rad_arr - (mean_vi + 1.5 * std_vi)
ndfi_window_arr = ndfi_arr - (mean_nd + 1.5 * std_nd)
btd_window_arr = btd_arr - (mean_btd + 1.5 * std_btd)
mir_window_arr = mir_bt_arr - (mean_mir + 1.5 * std_mir)
visdif_window_arr = vi_diff_arr - (mean_vid + 1.5 * std_vid)

x1 = 0.02
x2 = 8.

t1 = 270
t2 = 300

grad = (x2 - x1) / (t2 - t1)

lw_std_mult = grad * (lwi_bt_arr - t1)
lw_std_mult = np.where(lw_std_mult > x2, x2, lw_std_mult)
lw_std_mult = np.where(lw_std_mult < x1, x1, lw_std_mult)

lw_window_arr =  mean_lw - lw_std_mult * std_lw

In [7]:
save_output(scn, lw_std_mult, 'lw_std_mult', odir, 'VI1_DIFF')

In [8]:
save_output(scn, mean_nd, 'mean_nd', odir, 'VI1_DIFF')
save_output(scn, std_nd, 'std_nd', odir, 'VI1_DIFF')
save_output(scn, mean_lw, 'mean_lw', odir, 'VI1_DIFF')
save_output(scn, std_lw, 'std_lw', odir, 'VI1_DIFF')
save_output(scn, mean_btd, 'mean_btd', odir, 'VI1_DIFF')
save_output(scn, std_btd, 'std_btd', odir, 'VI1_DIFF')
save_output(scn, mean_vid, 'mean_vid', odir, 'VI1_DIFF')
save_output(scn, std_vid, 'std_vid', odir, 'VI1_DIFF')
save_output(scn, mean_mir, 'mean_mir', odir, 'VI1_DIFF')
save_output(scn, std_mir, 'std_mir', odir, 'VI1_DIFF')

In [30]:
sza_arr = np.array(scn['SZA'])

In [31]:
gd_t4 = mean_mir + 2 * std_mir
gd_btd1 = mean_btd + 2.5
gd_btd2 = mean_btd + 2 * std_btd

btd_thresh = np.where(sza_arr > 70, 3, 5)

lwbt_mult = (lwi_bt_arr - 270) / 30
lwbt_mult = np.where(lwbt_mult > 1, 1, lwbt_mult)
lwbt_mult = np.where(lwbt_mult < 0, 0, lwbt_mult)
lwbt_mult = 1. / (0.25 + 0.75 * lwbt_mult)

sza_mult = np.cos(np.deg2rad(sza_arr))
sza_mult = 0.4 + ((90 - sza_arr) / 90.)
ndfi_thresh = 0.001 + ((0.006 * lwbt_mult) + (0.003 / perc_good)) * sza_mult

#save_output(scn, gd_t4, 'gd_t4', odir, 'VI1_DIFF')
#save_output(scn, gd_btd1, 'gd_btd1', odir, 'VI1_DIFF')
#save_output(scn, gd_btd2, 'gd_btd2', odir, 'VI1_DIFF')
#save_output(scn, lwbt_mult, 'lwbt_mult', odir, 'VI1_DIFF')
#save_output(scn, ndfi_thresh, 'ndfi_thresh', odir, 'VI1_DIFF')
#save_output(scn, sza_mult, 'sza_mult', odir, 'VI1_DIFF')

In [10]:
save_output(scn, vis_window_arr, 'vis_window_arr', odir, 'VI1_DIFF')
save_output(scn, ndfi_window_arr, 'ndfi_window_arr', odir, 'VI1_DIFF')
save_output(scn, lw_window_arr, 'lw_window_arr', odir, 'VI1_DIFF')
save_output(scn, btd_window_arr, 'btd_window_arr', odir, 'VI1_DIFF')
save_output(scn, mir_window_arr, 'mir_window_arr', odir, 'VI1_DIFF')
save_output(scn, visdif_window_arr, 'visdif_window_arr', odir, 'VI1_DIFF')
save_output(scn, perc_good, 'perc_good', odir, 'VI1_DIFF')

In [38]:
outarr = da.where(pfp_arr > 0,                      1, 0) #t1
outarr = da.where(ndfi_window_arr > ndfi_thresh, outarr+1, outarr) #t2
outarr = da.where(visdif_window_arr > 0,         outarr+1, outarr) #t3
outarr = da.where(vis_window_arr < 0.1,          outarr+1, outarr) #t4
outarr = da.where(lwi_bt_arr > 260,              outarr+1, outarr) #t5
outarr = da.where(mir_window_arr > 0,            outarr+1, outarr) #t6
outarr = da.where(lsm == 2,                      outarr+1, outarr) #t7
outarr = da.where(mir_bt_arr > gd_t4,            outarr+1, outarr) #t8
outarr = da.where(btd_arr > gd_btd1,             outarr+1, outarr) #t9
outarr = da.where(btd_arr > gd_btd2,             outarr+1, outarr) #t10
outarr = da.where(lwi_bt_arr > lw_window_arr,    outarr+1, outarr) #t11
outarr = da.where(perc_good > 0.4,               outarr,   0) #t12
outarr = da.where(pfp_arr > 0,               outarr,   0) #t1

save_output(scn, outarr, 'outarr', tod, 'VI1_DIFF')

In [12]:
outarr1 = da.where(stg1_output > 0, 1, 0)
save_output(scn, outarr1, 't1', odir, 'VI1_DIFF')

outarr1 = da.where(ndfi_window_arr > ndfi_thresh, 1, 0)
save_output(scn, outarr1, 't2', odir, 'VI1_DIFF')

outarr1 = da.where(visdif_window_arr > 0, 1, 0)
save_output(scn, outarr1, 't3', odir, 'VI1_DIFF')

outarr1 = da.where(vis_window_arr < 0.1, 1, 0)
save_output(scn, outarr1, 't4', odir, 'VI1_DIFF')

outarr1 = da.where(lwi_bt_arr > 260, 1, 0)
save_output(scn, outarr1, 't5', odir, 'VI1_DIFF')

outarr1 = da.where(mir_window_arr > 0, 1, 0)
save_output(scn, outarr1, 't6', odir, 'VI1_DIFF')

outarr1 = da.where(lsm == 2, 1, 0)
save_output(scn, outarr1, 't7', odir, 'VI1_DIFF')

outarr1 = da.where(mir_bt_arr > gd_t4, 1, 0)
save_output(scn, outarr1, 't8', odir, 'VI1_DIFF')

outarr1 = da.where(btd_arr > gd_btd1, 1, 0)
save_output(scn, outarr1, 't9', odir, 'VI1_DIFF')

outarr1 = da.where(btd_arr > gd_btd2, 1, 0)
save_output(scn, outarr1, 't10', odir, 'VI1_DIFF')

outarr1 = da.where(lwi_bt_arr > lw_window_arr, 1, 0)
save_output(scn, outarr1, 't11', odir, 'VI1_DIFF')

outarr1 = da.where(perc_good > 0.2, 1, 0)
save_output(scn, outarr1, 't12', odir, 'VI1_DIFF')

In [13]:
outarr_main = da.where(outarr >= 10, 1, 0)
save_output(scn, outarr_main, 'outarr_main', odir, 'VI1_DIFF')

In [14]:
save_output(scn, n_winpix, 'n_winpix', odir, 'VI1_DIFF')
save_output(scn, n_cloudpix, 'n_cloudpix', odir, 'VI1_DIFF')
save_output(scn, n_waterpix, 'n_waterpix', odir, 'VI1_DIFF')

In [17]:
confidence = do_stage5(btd_arr, mean_btd, std_btd, mir_bt_arr, mean_mir, std_mir, 
                       sza_arr, n_winpix, n_cloudpix, n_waterpix)
confidence = np.where(perc_good > 0.6, confidence, 0)
save_output(scn, confidence, 'confidence', odir, 'VI1_DIFF')

In [26]:
med_btd = median_filter(da.array(btd_arr), size=5)
med_b13 = median_filter(scnbob['B13'].data, size=5)
med_b10 = median_filter(scnbob['B10'].data, size=5)
med_b07 = median_filter(scnbob['B07'].data, size=5)
med_b06 = median_filter(scnbob['B06'].data, size=5)

save_output(scn, med_btd, 'med_btd', odir, 'VI1_DIFF')
save_output(scn, med_b13, 'med_b13', odir, 'VI1_DIFF')
save_output(scn, med_b10, 'med_b10', odir, 'VI1_DIFF')
save_output(scn, med_b07, 'med_b07', odir, 'VI1_DIFF')
save_output(scn, med_b06, 'med_b06', odir, 'VI1_DIFF')

In [32]:
#thr_btd = threshold_local(da.array(btd_arr),  block_size=19)
#thr_b13 = threshold_local(scnbob['B13'].data, block_size=19)
#thr_b10 = threshold_local(scnbob['B10'].data, block_size=19)
#thr_b07 = threshold_local(scnbob['B07'].data, block_size=19)
thr_b06 = threshold_local(scnbob['B06'].data, block_size=59)

#save_output(scn, thr_btd, 'thr_btd', odir, 'VI1_DIFF')
#save_output(scn, thr_b13, 'thr_b13', odir, 'VI1_DIFF')
#save_output(scn, thr_b10, 'thr_b10', odir, 'VI1_DIFF')
#save_output(scn, thr_b07, 'thr_b07', odir, 'VI1_DIFF')
save_output(scn, thr_b06, 'thr_b06', odir, 'VI1_DIFF')

In [59]:
save_output(scn, mir_bt_arr, 'mir_bt_arr', tod, 'VI1_DIFF')

In [4]:
%%cython

from libc.math cimport sqrt, isnan, isfinite
cimport numpy as np
import numpy as np
import cython

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
@cython.initializedcheck(False)
cdef get_curwindow_pos(int xpos, int ypos, int winsize, int scn_width, int scn_height):
    """Find the correct position of the moving window in the input arrays.
    Inputs:
     - xpos: The x position of the candidate pixel
     - ypos: The y position of the candidate pixel
     - winsize: The radius of the window
     - scn_width: The width of the whole satellite scene
     - scn_height: The height of the whole satellite scene 
    Returns:
     - shape_sel: An array specifying the correct indices for the windowed region (x_0, x_1, y_0, y_1)
    """
    cdef int shape_sel[4]

    # Set up the output pixel selection
    shape_sel = [xpos - winsize, xpos + winsize + 1, ypos - winsize, ypos + winsize + 1]

    # Apply some range tests, so we can use cython in the non-bounds checking mode
    if shape_sel[0] < 0:
        shape_sel[0] = -999
    elif shape_sel[0] >= scn_width:
        shape_sel[0] = -999
    if shape_sel[1] < 0:
        shape_sel[0] = -999
    elif shape_sel[1] >= scn_height:
        shape_sel[0] = -999

    return shape_sel

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
@cython.initializedcheck(False)
cdef get_window_mea_stdv(int winsize,
                         float[:,:] cur__vis,
                         float[:,:] cur_ndfi,
                         float[:,:] cur__lwi,
                         float[:,:] cur__btd,
                         float[:,:] cur__mir,
                         float[:,:] cur__vid,
                         unsigned char[:,:] cur__lsm,
                         float cen__mir,
                         float cen__btd,
                         unsigned char lsm_land_val = 2,
                         float mir_min_thresh = 275,
                         float mir_max_thresh = 340,
                         float btd_max_thresh = 7,
                         float lwi_min_thresh = 263.,
                         ):
    cdef int arrsize = winsize + winsize + 1
    cdef int npix = arrsize * arrsize
    cdef int i = 0
    cdef int j = 0

    cdef float n_good = 0
    cdef float perc_good = 0

    cdef float sum_val_vi = 0
    cdef float mea_val_vi = -999.
    cdef float std_val_vi = -999.

    cdef float sum_val_nd = 0
    cdef float mea_val_nd = -999.
    cdef float std_val_nd = -999.

    cdef float sum_val_lw = 0
    cdef float mea_val_lw = -999.
    cdef float std_val_lw = -999.

    cdef float sum_val_btd = 0
    cdef float mea_val_btd = -999.
    cdef float std_val_btd = -999.

    cdef float sum_val_mir = 0
    cdef float mea_val_mir = -999.
    cdef float std_val_mir = -999.

    cdef float sum_val_vid = 0
    cdef float mea_val_vid = -999.
    cdef float std_val_vid = -999.

    cdef float res[15]

    cdef float n_wat = 0
    cdef float n_cld = 0


    # Loop to find mean
    for i in range(0, arrsize):
        for j in range(0, arrsize):
            if i >= winsize - 1 and i <= winsize + 1 and j >= winsize - 1 and j <= winsize + 1:
                continue
            if cur__lwi[i,j] < lwi_min_thresh or isnan(cur__lwi[i, j]):
                n_cld = n_cld + 1
                continue
            if cur__mir[i,j] > cen__mir or isnan(cur__mir[i, j]):
                continue
            if cur__btd[i,j] > cen__btd or isnan(cur__btd[i, j]):
                continue
            if cur__mir[i,j] < mir_min_thresh or cur__mir[i,j] > mir_max_thresh:
                continue
            if cur__btd[i,j] > btd_max_thresh:
                continue
            if lsm_land_val != 255:
                if cur__lsm[i,j] != lsm_land_val:
                    n_wat = n_wat + 1
                    continue
            sum_val_vi = sum_val_vi + cur__vis[i, j]
            sum_val_nd = sum_val_nd + cur_ndfi[i, j]
            sum_val_lw = sum_val_lw + cur__lwi[i, j]
            sum_val_btd = sum_val_btd + cur__btd[i, j]
            sum_val_mir = sum_val_mir + cur__mir[i, j]
            sum_val_vid = sum_val_vid + cur__vid[i, j]
            n_good = n_good + 1
    perc_good = n_good / (npix - 9)

    mea_val_vi = sum_val_vi / n_good
    mea_val_nd = sum_val_nd / n_good
    mea_val_lw = sum_val_lw / n_good
    mea_val_btd = sum_val_btd / n_good
    mea_val_mir = sum_val_mir / n_good
    mea_val_vid = sum_val_vid / n_good

     # Loop to find standard deviation
    sum_val_vi = 0
    sum_val_nd = 0
    sum_val_lw = 0
    sum_val_btd = 0
    sum_val_mir = 0
    sum_val_vid = 0

    for i in range(0, arrsize):
        for j in range(0, arrsize):
            if i >= winsize - 1 and i <= winsize + 1 and j >= winsize - 1 and j <= winsize + 1:
                continue
            if cur__mir[i,j] > cen__mir or isnan(cur__mir[i, j]):
                continue
            if cur__btd[i,j] > cen__btd or isnan(cur__btd[i, j]):
                continue
            if cur__mir[i,j] < mir_min_thresh or cur__mir[i,j] > mir_max_thresh:
                continue
            if cur__btd[i,j] > btd_max_thresh:
                continue
            if cur__lwi[i,j] < lwi_min_thresh or isnan(cur__lwi[i, j]):
                continue
            if lsm_land_val != 255:
                if cur__lsm[i,j] != lsm_land_val:
                    continue
            sum_val_vi = sum_val_vi + (cur__vis[i, j] - mea_val_vi) * (cur__vis[i, j] - mea_val_vi)
            sum_val_nd = sum_val_nd + (cur_ndfi[i, j] - mea_val_nd) * (cur_ndfi[i, j] - mea_val_nd)
            sum_val_lw = sum_val_lw + (cur__lwi[i, j] - mea_val_lw) * (cur__lwi[i, j] - mea_val_lw)
            sum_val_btd = sum_val_btd + (cur__btd[i, j] - mea_val_btd) * (cur__btd[i, j] - mea_val_btd)
            sum_val_mir = sum_val_mir + (cur__mir[i, j] - mea_val_mir) * (cur__mir[i, j] - mea_val_mir)
            sum_val_vid = sum_val_vid + (cur__vid[i, j] - mea_val_vid) * (cur__vid[i, j] - mea_val_vid)

    std_val_lw = sqrt(sum_val_lw / n_good)
    std_val_nd = sqrt(sum_val_nd / n_good)
    std_val_vi = sqrt(sum_val_vi / n_good)
    std_val_btd = sqrt(sum_val_btd / n_good)
    std_val_mir = sqrt(sum_val_mir / n_good)
    std_val_vid = sqrt(sum_val_vid / n_good)

    res[0] = perc_good
    res[1] = n_cld
    res[2] = n_wat
    res[3] = mea_val_lw
    res[4] = std_val_lw
    res[5] = mea_val_nd
    res[6] = std_val_nd
    res[7] = mea_val_vi
    res[8] = std_val_vi
    res[9] = mea_val_btd
    res[10] = std_val_btd
    res[11] = mea_val_mir
    res[12] = std_val_mir
    res[13] = mea_val_vid
    res[14] = std_val_vid
    return res


def get_mea_std_window(unsigned char[:,:] pfp_data,
                       float[:,:] vi_rad_data,
                       float[:,:] ndfi_data,
                       float[:,:] lw_rad_data,
                       float[:,:] btd_arr,
                       float[:,:] mir_arr,
                       float[:,:] vid_arr,
                       unsigned char[:,:] lsm,
                       unsigned char lsm_land_val,
                       int winsize):

    cdef int scn_width = vi_rad_data.shape[0]
    cdef int scn_height = vi_rad_data.shape[1]

    cdef int x_0 = 0
    cdef int x_1 = scn_width
    cdef int y_0 = 0
    cdef int y_1 = scn_height

    # Counters for main loop
    cdef int x = 0
    cdef int y = 0
    cdef int wsize = winsize
    cdef int cur_win[4]

    # Intermediate variables
    cdef float sum_tot = 0
    cdef int n = 0
    cdef float cen_vid = 0
    cdef float cen_btd = 999
    cdef float cen_mir = 999
    cdef float cen_irr = 0
    cdef float cen_lwi = 0
    cdef float res[15]

    # Output datasets
    cdef np.ndarray[dtype=np.float32_t, ndim=3] outarr = np.zeros((16, scn_width, scn_height), dtype=np.single)
    cdef float[:,:, ::1] outarr_view = outarr

    min_wsize = 5
    max_wsize = 15
    perc_thresh = 0.65

    # Loop across all pixels in the image
    for x in range(x_0, x_1):
        for y in range(y_0, y_1):
            if pfp_data[x, y] != 1:
                outarr_view[:, x, y] = -999
                continue
            for wsize in range(min_wsize, max_wsize):
                # Determine coordinates of the current window
                cur_win = get_curwindow_pos(x, y, wsize, scn_width, scn_height)
                # Don't process if any coords are bad (usually at extreme edges of image)
                if cur_win[0] < 0:
                    continue

                cur__vis = vi_rad_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur_ndfi = ndfi_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur__lwi = lw_rad_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur__btd = btd_arr[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur__mir = mir_arr[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur__vid = vid_arr[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur__lsm = lsm[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]

                cen_mir = mir_arr[x, y]
                cen_btd = btd_arr[x, y]

                # Compute mean and standard deviation across window
                res = get_window_mea_stdv(wsize,
                                          cur__vis,
                                          cur_ndfi,
                                          cur__lwi,
                                          cur__btd,
                                          cur__mir,
                                          cur__vid,
                                          cur__lsm,
                                          cen_mir,
                                          cen_btd,
                                          lsm_land_val)
                #print(f'{res[0]:4.4f} {res[1]:4.4f} {res[2]:4.4f} {res[3]:4.4f} {res[4]:4.4f} {res[5]:4.4f}{res[6]:4.4f} {res[7]:4.4f} {res[8]:4.4f} {res[9]:4.4f} {res[10]:4.4f} {res[11]:4.4f} {res[12]:4.4f} {res[13]:4.4f} {res[14]:4.4f} {res[15]:4.4f}')
                if res[0] > perc_thresh:
                    break

            # Set output array to:
            # [0]: Percentage of acceptable pixels in window
            outarr_view[0, x, y] = res[0]            # Percentage of good pixels
            outarr_view[1, x, y] = wsize * wsize     # Number of pixels in window
            outarr_view[2, x, y] = res[1]            # Number of cloudy pixels in window
            outarr_view[3, x, y] = res[2]            # Number of water body pixels in window
            outarr_view[4, x, y] = res[3]            # LW BT mean
            outarr_view[5, x, y] = res[4]            # LW BT std-dev
            outarr_view[6, x, y] = res[5]            # NDFI mean
            outarr_view[7, x, y] = res[6]            # NDFI std-dev
            outarr_view[8, x, y] = res[7]            # VIS radiance mean
            outarr_view[9, x, y] = res[8]            # VIS radiance std-dev
            outarr_view[10, x, y] = res[9]           # BTD mean
            outarr_view[11, x, y] = res[10]          # BTD std-dev
            outarr_view[12, x, y] = res[11]          # MIR BT mean
            outarr_view[13, x, y] = res[12]          # MIR BT std-dev
            outarr_view[14, x, y] = res[13]          # VisDiff mean
            outarr_view[15, x, y] = res[14]          # VisDiff std-dev

    return outarr

In [7]:
%load_ext Cython
%load_ext autoreload
%autoreload 2

from dask_image.ndfilters import median_filter, threshold_local, percentile_filter
from cython.parallel import prange

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from datetime import datetime
from satpy import Scene
import dask.array as da
from glob import glob

from pyfires.PYF_detection import stage1_tests, run_basic_night_detection
import pyfires.PYF_Consts as PYFc
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from satpy import Scene
from glob import glob
import os
bdict = {'vi1_band': 'B03',
         'vi2_band': 'B06',
         'mir_band': 'B07',
         'lwi_band': 'B13'}


curfile = 'D:/sat_data/ahi_main/in/0840/HS_H09_20231029_0840_B07_FLDK_R20_S0110.DAT'
tod = 'D:/sat_data/ahi_main/out/'
pos = curfile.find('B07')
dtstr = curfile[pos-14:pos-1]
ifiles_l15 = glob(f'{os.path.dirname(curfile)}/*{dtstr}*.DAT')
scn = initial_load(ifiles_l15,  # List of files to load
                       'ahi_hsd',   # The reader to use, in this case the AHI HSD reader
                       bdict,       # The bands to load
                       mir_bt_thresh=270, # The threshold for the MIR band, pixels cooler than this are excluded.
                       lw_bt_thresh=260)

#scn.save_datasets(base_dir=tod, enhance=False, dtype=np.float32)

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


cannot convert float NaN to integer


In [8]:
scn = stage1_tests(scn, ksizes=[5, 7, 9], do_lsm_mask=False)

opts = {'def_fire_rad_vis': 0.5,
        'def_fire_rad_vid': 0.1,
        'sza_thresh': 97,
        'vid_thresh': 0.02
        }

#scn.save_dataset('PFP', enhance=False, dtype=np.float32, base_dir=tod)

night_res = run_basic_night_detection(scn, opts)

In [9]:
from dask_image.ndfilters import sobel

kern_x = np.array(([1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]))
kern_y = np.array(([1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]))

kern_ = np.ones((3, 3))
kern_ = kern_ / np.sum(kern_)

sob_x_lw = convolve(scn['LW1_RAD'].data, kern_x)
sob_y_lw = convolve(scn['LW1_RAD'].data, kern_y)
sob_a_lw = convolve(scn['LW1_RAD'].data, kern_)
sob_lw = np.sqrt(sob_x_lw * sob_x_lw + sob_y_lw * sob_y_lw) / sob_a_lw

sob_x_mi = convolve(scn['MIR_RAD'].data, kern_x)
sob_y_mi = convolve(scn['MIR_RAD'].data, kern_y)
sob_a_mi = convolve(scn['MIR_RAD'].data, kern_)
sob_mi = np.sqrt(sob_x_mi * sob_x_mi + sob_y_mi * sob_y_mi) / sob_a_mi

sob_x_vi = convolve(scn['VI1_RAD'].data, kern_x)
sob_y_vi = convolve(scn['VI1_RAD'].data, kern_y)
sob_a_vi = convolve(scn['VI1_RAD'].data, kern_)
sob_vi = np.sqrt(sob_x_vi * sob_x_vi + sob_y_vi * sob_y_vi) / sob_a_vi
sob_vi = np.where(scn['VI1_RAD'] < 0.001, 0, sob_vi)

#save_output(scn, sob_lw, 'sob_lw', tod, 'VI1_DIFF')
#save_output(scn, sob_mi, 'sob_mi', tod, 'VI1_DIFF')
#save_output(scn, sob_vi, 'sob_vi', tod, 'VI1_DIFF')
#save_output(scn, sob_lw2, 'sob_lw2', tod, 'VI1_DIFF')
#save_output(scn, sob_mi2, 'sob_mi2', tod, 'VI1_DIFF')
#save_output(scn, sob_vi2, 'sob_vi2', tod, 'VI1_DIFF')

In [10]:
sobdiff = sob_mi - (sob_vi + sob_lw)
#save_output(scn, sobdiff, 'sobdiff', tod, 'VI1_DIFF')

In [14]:
sob_ndfi = np.where(sobdiff > 0., 1, 0)
#save_output(scn, sob_ndfi, 'sob_ndfi', tod, 'VI1_DIFF')
sob_ndfi = np.where(scn['PFP'] > 0., sob_ndfi, 0)
sob_ndfi = np.where(scn['SZA'] > 97, 0, sob_ndfi)
#save_output(scn, sob_ndfi, 'sob_ndfi2', tod, 'VI1_DIFF')

In [11]:
outarr = np.where(sob_mi > sob_lw, 1, 0)
outarr = np.where(sob_mi2 > sob_lw2, outarr, 0)
save_output(scn, outarr, 'dets1', tod, 'VI1_DIFF')
outarr = np.where(scn['PFP'].data > 0, outarr, 0)
save_output(scn, outarr, 'dets2', tod, 'VI1_DIFF')


NameError: name 'sob_mi2' is not defined

In [140]:
visr = np.array(scn['VI1_RAD'])
mirr = np.array(scn['MIR_RAD'])
lwir = np.array(scn['LW1_RAD'])

print(np.nanmin(visr), np.nanpercentile(visr, 5), np.nanmean(visr), np.nanpercentile(visr, 95), np.nanmax(visr))
print(np.nanmin(mirr), np.nanpercentile(mirr, 5), np.nanmean(mirr), np.nanpercentile(mirr, 95), np.nanmax(mirr))
print(np.nanmin(lwir), np.nanpercentile(lwir, 5), np.nanmean(lwir), np.nanpercentile(lwir, 95), np.nanmax(lwir))

0.0 0.0 0.16270022 0.8574572801589966 2.636805
-0.005914688 0.041405677795410156 0.3429453 0.6151657104492188 12.088894
0.51034355 2.7130651473999023 7.0443487 9.47508430480957 12.004273


In [133]:
sza_mult_min = 87
sza_mult_max = 97

daynight_norm_thresh = 0.01

mult = 0.1 * (scn['SZA'] - 87)

mult = mult.clip(0, 1)

def do_normed(indata, rsize):
    kern = np.ones((rsize, rsize))
    kern = kern / np.sum(kern)
    dn_ave = convolve(indata, kern)
    differ = indata - dn_ave
    return differ# / indata
    

daynight = mult * scn['RAD_ADD'] + (1. - mult) * scn['VI1_DIFF']
daynight = daynight.clip(0, 100)

daynight_normed_07 = do_normed(daynight.data, 7)
daynight_normed_21 = do_normed(daynight.data, 21)

scn['PFP'].data = np.where(daynight_normed_21 > daynight_norm_thresh, scn['PFP'].data, 0)

In [134]:
scn.save_dataset('PFP', enhance=False, dtype=np.float32, base_dir=tod)

In [22]:
pfp_arr = np.array(sob_ndfi).astype(np.uint8)
vi_rad_arr = np.array(scn['VI1_RAD'])
ndfi_arr = np.array(scn['mi_ndfi'])
vi_diff_arr = np.array(scn['VI1_DIFF'])
btd_arr = np.array(scn['BTD']) 
mir_bt_arr = np.array(scn['MIR__BT'])
irr_arr = np.array(scn['VI2_RAD'])
lwi_bt_arr = np.array(scn['LW1__BT'])
lsm = np.array(scn['LSM'])
#night_res = np.array(results).astype(np.uint8)

In [34]:
pfp_arr = np.array(sob_ndfi).astype(np.uint8)

In [23]:
# This section computes windowed statistics around each candidate fire pixel.

from pyfires import PYF_WindowStats as PYFW

st_time = datetime.utcnow()
outa = PYFW.get_mea_std_window(pfp_arr,       # Potential fire pixel candidates
                               vi_rad_arr,    # VIS chan
                               ndfi_arr,      # NDFI
                               lwi_bt_arr,    # LW Brightness Temperature
                               btd_arr,       # MIR-LW BTD
                               mir_bt_arr,    # MIR BT
                               vi_diff_arr,   # MIR-LWIR-VIS radiance diff
                               lsm,           # The land-sea mask
                               255,           # The value denoting land in the LSM. If 255, ignore mask
                               25)
en_time = datetime.utcnow()

print((en_time - st_time).total_seconds())

3.153506


In [26]:
# Get the results of the windows statistics code

#outan = da.from_delayed(outa, shape=(scn['BTD'].shape[0], scn['BTD'].shape[1], 16), dtype=np.single)

perc_good  = np.squeeze(outa[0,:,:]); n_winpix   = np.squeeze(outa[1,:,:])
n_cloudpix = np.squeeze(outa[2,:,:]); n_waterpix = np.squeeze(outa[3,:,:])
mean_lw    = np.squeeze(outa[4,:,:]); std_lw     = np.squeeze(outa[5,:,:])
mean_nd    = np.squeeze(outa[6,:,:]); std_nd     = np.squeeze(outa[7,:,:])
mean_vi    = np.squeeze(outa[8,:,:]); std_vi     = np.squeeze(outa[9,:,:])
mean_btd   = np.squeeze(outa[10,:,:]); std_btd    = np.squeeze(outa[11,:,:])
mean_mir   = np.squeeze(outa[12,:,:]); std_mir    = np.squeeze(outa[13,:,:])
mean_vid   = np.squeeze(outa[14,:,:]); std_vid    = np.squeeze(outa[15,:,:])

# Apply some additional thresholding to remove false positives due to VIS channel noise
night_res = da.where(scn['BTD'].data > mean_btd + 1.5 * std_btd, night_res, 0)
night_res = da.where(scn['VI1_DIFF'].data > mean_vid + 1.5 * std_vid, night_res, 0)

In [81]:
tod = 'D:/sat_data/ahi_main/out/'

save_output(scn, mean_vid, 'mean_vid', tod, 'VI1_DIFF')
save_output(scn, std_vid, 'std_vid', tod, 'VI1_DIFF')

save_output(scn, mean_btd, 'mean_btd', tod, 'VI1_DIFF')
save_output(scn, std_btd, 'std_btd', tod, 'VI1_DIFF')

save_output(scn, mean_mir, 'mean_mir', tod, 'VI1_DIFF')
save_output(scn, std_mir, 'std_mir', tod, 'VI1_DIFF')

#save_output(scn, night_res, 'extra_tests', tod, 'VI1_DIFF')

save_output(scn, perc_good, 'perc_good', tod, 'VI1_DIFF')

In [82]:
a_val = PYFc.rad_to_bt_dict[scn['pix_area'].attrs['platform_name']]

frp_est = (scn['pix_area'] * PYFc.sigma / a_val) * (scn['MIR__BT'] - mean_mir)
frp_est = np.where(mean_mir > 0, frp_est, 0)

In [83]:
save_output(scn, frp_est, 'frp_est', tod, 'VI1_DIFF')

In [27]:
save_output(scn, night_res, 'night_res', tod, 'VI1_DIFF')

In [119]:
ifiles_l15 = glob(f'{os.path.dirname(curfile)}/*{dtstr}*.DAT')
bscn = Scene(ifiles_l15, reader='ahi_hsd')
bscn.load(['B15', 'B13'], calibration='radiance')

bscn['diff'] = bscn['B13'].copy()
bscn['diff'].data = bscn['B13'].data - bscn['B15'].data
bscn['diff'].attrs['name'] = 'diff'
bscn.save_dataset('diff', base_dir=tod, enhance=False, dtype=np.float32)